In [1]:
import tensorflow.compat.v1 as tf  # TensorFlow v1 compatibility for v2, use "import tensorflow as tf" in v1

tf.disable_v2_behavior()  # disable TensorFlow v2. Delete if using v1
import cv2  # read in pixel data
import snake  # game class
import numpy as np
import random
import os
from collections import (
    deque,
)  # queue data structure. fast appends. and pops. replay memory
from numpy.random import choice  # to choose based on a probability

Instructions for updating:
non-resource variables are not supported in the long term
pygame 2.6.1 (SDL 2.28.4, Python 3.10.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Hyperparameters

USE_MODEL = False  # False = training, True = testing

ACTIONS = 5  # up, down, left, right, stay
GAMMA = 0.99  # learning rate

# Initial and final epsilon to balance exploration/exploitation
INITIAL_EPSILON = 1.0
FINAL_EPSILON = 0.05
EXPLORE = 10000  # how many frames until final epsilon is reached
OBSERVE = 1000  # how many frames to observe at training start

SAVE_STEP = 5000  # how many steps until save checkpoint
REPLAY_MEMORY = 200000  # store our experiences, the size of it

IMG_SIZE = 60  # images size to train on
BATCH = 64  # batch size to train on

In [3]:
# Create TensorFlow graph
def createGraph():
    """
    Network Architecture:

    Layer   Input       Filter size Stride  Num filters Activation	Output
    conv1   60x60x4     6×6	            4   32          ReLU        15x15x32
    pool1   15x15x32    2×2	            2                           8x8x32
    conv2   8x8x32      4×4	            2   64          ReLU        4x4x64
    conv3   4x4x64      3×3	            1   64          ReLU        4x4x64
    fc4     4x4x64                          512         ReLU        512
    fc5     512                             ACTIONS     Linear      ACTIONS

    """

    W_conv1 = tf.Variable(tf.truncated_normal([6, 6, 4, 32], stddev=0.02))
    b_conv1 = tf.Variable(tf.constant(0.01, shape=[32]))

    W_conv2 = tf.Variable(tf.truncated_normal([4, 4, 32, 64], stddev=0.02))
    b_conv2 = tf.Variable(tf.constant(0.01, shape=[64]))

    W_conv3 = tf.Variable(tf.truncated_normal([3, 3, 64, 64], stddev=0.02))
    b_conv3 = tf.Variable(tf.constant(0.01, shape=[64]))

    W_fc4 = tf.Variable(tf.truncated_normal([1024, 512], stddev=0.02))
    b_fc4 = tf.Variable(tf.constant(0.01, shape=[512]))

    W_fc5 = tf.Variable(tf.truncated_normal([512, ACTIONS], stddev=0.02))
    b_fc5 = tf.Variable(tf.constant(0.01, shape=[ACTIONS]))

    # input for pixel data
    s = tf.placeholder("float", [None, IMG_SIZE, IMG_SIZE, 4])

    conv1 = tf.nn.relu(
        tf.nn.conv2d(s, W_conv1, strides=[1, 4, 4, 1], padding="SAME") + b_conv1
    )

    pool1 = tf.nn.max_pool(
        conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME"
    )

    conv2 = tf.nn.relu(
        tf.nn.conv2d(pool1, W_conv2, strides=[1, 2, 2, 1], padding="SAME") + b_conv2
    )

    conv3 = tf.nn.relu(
        tf.nn.conv2d(conv2, W_conv3, strides=[1, 1, 1, 1], padding="SAME") + b_conv3
    )

    conv3_flat = tf.reshape(conv3, [-1, 1024])

    fc4 = tf.nn.relu(tf.matmul(conv3_flat, W_fc4) + b_fc4)

    fc5 = tf.matmul(fc4, W_fc5) + b_fc5

    return s, fc5

In [4]:
# Train deep q network. feed in pixel data to graph session
def trainGraph(inp, out):

    # to calculate the argmax, we multiply the predicted output with a vector with one value 1 and rest as 0
    argmax = tf.placeholder("float", [None, ACTIONS])
    gt = tf.placeholder("float", [None])  # ground truth
    global_step = tf.Variable(0, name="global_step")

    # action
    action = tf.reduce_sum(tf.multiply(out, argmax), reduction_indices=1)
    # cost function we will reduce through backpropagation
    cost = tf.reduce_mean(tf.square(action - gt))
    # optimization function to minimize our cost function
    train_step = tf.train.AdamOptimizer(1e-6).minimize(cost)

    # initialize our game
    game = snake.SnakeGame()

    # create a queue for experience replay to store policies
    D = deque()

    # intial frame
    frame = game.GetPresentFrame()
    # convert rgb to gray scale for processing
    frame = cv2.cvtColor(cv2.resize(frame, (IMG_SIZE, IMG_SIZE)), cv2.COLOR_BGR2GRAY)
    # binary colors, black or white
    ret, frame = cv2.threshold(frame, 1, 255, cv2.THRESH_BINARY)
    # stack frames, that is our input tensor
    inp_t = np.stack((frame, frame, frame, frame), axis=2)

    saver = tf.train.Saver(tf.global_variables())

    sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

    # restore latest checkpoint or initialize a new graph
    checkpoint = tf.train.latest_checkpoint("./checkpoints")
    if checkpoint != None:
        print("Restore Checkpoint %s" % (checkpoint))
        saver.restore(sess, checkpoint)
        print("Model restored.")
    else:
        init = tf.global_variables_initializer()
        sess.run(init)
        print("Initialized new Graph")

    t = global_step.eval()
    c = 0

    epsilon = INITIAL_EPSILON

    # Training time
    while 1:
        # output tensor
        out_t = out.eval(feed_dict={inp: [inp_t]})[0]
        # argmax function
        argmax_t = np.zeros([ACTIONS])

        # exploration or exploitation
        if random.random() <= epsilon and not USE_MODEL:
            # make 0 the most choosen action for realistic randomness
            maxIndex = choice((0, 1, 2, 3, 4), 1, p=(0.80, 0.05, 0.05, 0.05, 0.05))
        else:
            maxIndex = np.argmax(out_t)
        argmax_t[maxIndex] = 1

        if epsilon > FINAL_EPSILON:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        mode = "observing"
        if t > OBSERVE:
            mode = "training"
        if USE_MODEL:
            mode = "model only"

        # move game forward by one frame, get reward and frame
        reward_t, frame = game.GetNextFrame(argmax_t, [t, np.max(out_t), epsilon, mode])
        # get frame pixel data
        frame = cv2.cvtColor(
            cv2.resize(frame, (IMG_SIZE, IMG_SIZE)), cv2.COLOR_BGR2GRAY
        )
        ret, frame = cv2.threshold(frame, 1, 255, cv2.THRESH_BINARY)
        frame = np.reshape(frame, (IMG_SIZE, IMG_SIZE, 1))
        # new input tensor
        inp_t1 = np.append(frame, inp_t[:, :, 0:3], axis=2)

        # add our input tensor, argmax tensor, reward and updated input tensor to replay memory
        D.append((inp_t, argmax_t, reward_t, inp_t1))

        # if we run out of replay memory, make room
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        # training iteration
        if c > OBSERVE and not USE_MODEL:

            # get random sample from our replay memory
            minibatch = random.sample(D, BATCH)

            inp_batch = [d[0] for d in minibatch]
            argmax_batch = [d[1] for d in minibatch]
            reward_batch = [d[2] for d in minibatch]
            inp_t1_batch = [d[3] for d in minibatch]

            gt_batch = []
            out_batch = out.eval(feed_dict={inp: inp_t1_batch})

            # add values to our batch
            for i in range(0, len(minibatch)):
                gt_batch.append(reward_batch[i] + GAMMA * np.max(out_batch[i]))

            # train on that
            train_step.run(
                feed_dict={gt: gt_batch, argmax: argmax_batch, inp: inp_batch}
            )

        # update our input tensor the the next frame
        inp_t = inp_t1
        t = t + 1
        c = c + 1

        # print our where we are after saving where we are
        if t % SAVE_STEP == 0 and not USE_MODEL:
            sess.run(global_step.assign(t))
            saver.save(sess, "./checkpoints/model.ckpt", global_step=t)

        print(
            "TIMESTEP",
            t,
            "/ EPSILON",
            epsilon,
            "/ ACTION",
            maxIndex,
            "/ REWARD",
            reward_t,
            "/ Q_MAX %e" % np.max(out_t),
        )

In [5]:
if not os.path.exists('./checkpoints'):
    os.makedirs('./checkpoints')
#input layer and output layer by creating graph
inp, out = createGraph()

In [ ]:
# train our graph on input and output with session variables
trainGraph(inp, out)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6

Restore Checkpoint ./checkpoints\model.ckpt-1930000
INFO:tensorflow:Restoring parameters from ./checkpoints\model.ckpt-1930000
Model restored.
TIMESTEP 1930001 / EPSILON 0.999905 / ACTION [0] / REWARD 0 / Q_MAX 1.225804e+00
TIMESTEP 1930002 / EPSILON 0.9998100000000001 / ACTION [0] / REWARD 0 / Q_MAX 1.376794e+00
TIMESTEP 1930003 / EPSILON 0.9997150000000001 / ACTION [0] / REWARD 0 / Q_MAX 1.293372e+00
TIMESTEP 1930004 / EPSILON 0.9996200000000002 / ACTION [0] / REWARD 0 / Q_MAX 1.343323e+00
TIMESTEP 1930005 / EPSILON 0.9995250000000002 / ACTION [0] / REWARD 0 / Q_MAX 1.360647e+00
TIMESTEP 1930006 / EPSILON 0.9994300000000003 / ACTION [0] / REWARD 0 / Q_MAX 1.368833e+00
TIMESTEP 1930007 / EPSILON 0.9993350000000003 / ACTION [0] / REWARD 0 / Q_MAX 1.377966e+00
TIMESTEP 1930008 / EPSILON 0.9992400000000004 / ACTION [0] / R